<a href="https://colab.research.google.com/github/javierdelapuente/bullet3/blob/master/calculo_esperanzas_rangos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cálculo de probabilidades de rangos de premios

In [1]:
import sys
import os
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
    
if IN_COLAB:
    # Esto es para la librería
    ! git clone https://github.com/javierdelapuente/quiniela.git
    ! cd quiniela && git pull
    nuevo_path = os.path.join(os.getcwd(), "quiniela")
    if nuevo_path not in sys.path:
        sys.path.insert(0, nuevo_path)  

fatal: destination path 'quiniela' already exists and is not an empty directory.
Already up to date.


In [2]:
import os
%matplotlib inline
import matplotlib.pyplot as plt
import itertools
import numpy as np
import pandas as pd
import time
import datetime
import quiniela_utils as qu
from IPython.display import display
from scipy.special import comb
from scipy.stats import binom
np.set_printoptions(precision=25)

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [3]:
%%time
partidos_one_hot = qu.crear_todos_partidos()

CPU times: user 1.25 s, sys: 135 ms, total: 1.38 s
Wall time: 1.38 s


In [4]:
jornada = qu.DatosJornada(2021, 24)

probs_real = jornada.probabilidades_reales
probs_lae = jornada.probabilidades_lae
probs_real, probs_lae

(array([[0.42350000000000004, 0.2995             , 0.277              ],
        [0.7458             , 0.16140000000000002, 0.0928             ],
        [0.3375             , 0.33640000000000003, 0.3261             ],
        [0.3848             , 0.2788             , 0.33640000000000003],
        [0.4275             , 0.2863             , 0.2862             ],
        [0.6406999999999999 , 0.2427             , 0.11660000000000001],
        [0.5315             , 0.2621             , 0.2064             ],
        [0.09140000000000001, 0.16899999999999998, 0.7395999999999999 ],
        [0.4042             , 0.29600000000000004, 0.2998             ],
        [0.3907             , 0.2878             , 0.3215             ],
        [0.4073             , 0.3091             , 0.2836             ],
        [0.5945             , 0.23320000000000002, 0.1723             ],
        [0.5922999999999999 , 0.23809999999999998, 0.1696             ],
        [0.10060000000000001, 0.1666             , 

In [5]:
num_apuestas = 5
idx_partidos = np.random.randint(0, partidos_one_hot.shape[0], size=num_apuestas)
apuestas_jugadas = partidos_one_hot[idx_partidos]
apuestas_jugadas

array([[[0, 1, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0]],

       [[0, 0, 1],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1]],

       [[1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1]],

       [[0, 1, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 0, 1],
        [1, 0, 0],
      

In [6]:
from numpy import floor, ceil, log, exp, sqrt, log1p, expm1, tanh, cosh, sinh
from scipy.special import entr, logsumexp, betaln, gammaln as gamln
from scipy import special

# copiado de scipy.stats y optimizado para calcule el coef binomial por adelantado
class MiBinom:
    def __init__(self, n, max_k):
        self.n = n
        self.max_k = max_k
        self.k = np.arange(max_k)
        self.combiln = (gamln(self.n+1) - (gamln(self.k+1) + gamln(self.n-self.k+1)))
        
    def logpmf(self, p):
        return self.combiln + special.xlogy(self.k, p) + special.xlog1py(self.n-self.k, -p)

    def pmf(self, p):
        return exp(self.logpmf(p))


In [7]:
for num_aciertos in range(10, 15):
    # 1. reshape y elimino duplicados, con cuenta de duplicados de cada uno de ellos.
    partidos_ganadores = qu.generar_todas_combinaciones_ganadoras(apuestas_jugadas, num_aciertos)
    #display(partidos_ganadores.shape)
    partidos_ganadores = np.reshape(partidos_ganadores, (-1, partidos_ganadores.shape[-2],partidos_ganadores.shape[-1]))
    display(partidos_ganadores.shape)
    partidos_ganadores, cuenta = np.unique(partidos_ganadores, axis = 0, return_counts = True)
    display(partidos_ganadores.shape)

    # 2. calculo prob reales para cada uno de ellos.
    partidos_probs_reales = (partidos_ganadores*probs_real).sum(axis = 2).prod(axis = 1)
    # display(partidos_probs_reales.shape)
    
    # 3. calculo prob lae para cada uno de ellos. *en bloques de 20000 para que no pete la memoria.
    partidos_prob_lae = qu.probabilidades_lae_partidos(partidos_ganadores, num_aciertos, probs_lae)

    # 4. Esto es lo jodido. Explicarlo.
    a_repartir = jornada.total_premios_con_aciertos(num_aciertos)
    num_apuestas_jugadas = jornada.num_apuestas_jugadas

    # cuidado, que para 14 se reparte lo que sea...
    minimo_importe_repartido = (1.5 if num_aciertos == 10 else 1)
    b = MiBinom(num_apuestas_jugadas, a_repartir//1.5)

    start = time.time()
    for i, (p_real, p_lae) in enumerate(zip(partidos_probs_reales, partidos_prob_lae)):
      pmf = b.pmf(p_lae) # esto es lo lento...
      if i % 1000 == 0:
        end = time.time()
        print(i, "time", end - start)



(80080, 14, 3)

(79051, 14, 3)

DEBUG:root:0,10000 (10000, 14, 3)
DEBUG:root:10000,20000 (10000, 14, 3)
DEBUG:root:20000,30000 (10000, 14, 3)
DEBUG:root:30000,40000 (10000, 14, 3)
DEBUG:root:40000,50000 (10000, 14, 3)
DEBUG:root:50000,60000 (10000, 14, 3)
DEBUG:root:60000,70000 (10000, 14, 3)
DEBUG:root:70000,79051 (9051, 14, 3)


0 time 0.00826573371887207
1000 time 8.03368854522705


KeyboardInterrupt: ignored